In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from fpdf import FPDF


In [2]:
def fetch_financial_data(ticker, API_KEY):
    base_url = 'https://financialmodelingprep.com/api'
    data_type = 'income-statement'
    url = f'{base_url}/v3/{data_type}/{ticker}?apikey={API_KEY}'
    
    response = requests.get(url)
    data = response.json()
    
    df = pd.DataFrame(data)
    filtered_df = df[['calendarYear', 'revenue', 'grossProfit', 'ebitda', 'ebitdaratio', 'eps']]
    filtered_df = filtered_df.sort_values(by='calendarYear', ascending=False).head(5)
    filtered_df = filtered_df.sort_values(by='calendarYear') 
    return filtered_df

In [3]:
def generate_plots(filtered_df, ticker):
    plt.figure(figsize=(6, 4))
    plt.plot(filtered_df['calendarYear'], filtered_df['eps'], marker='o', color='b')
    plt.title(f'{ticker} EPS Trend Over Last 5 Years')
    plt.xlabel('Year')
    plt.ylabel('EPS')
    
    for i in range(len(filtered_df)):
        plt.text(filtered_df['calendarYear'].iloc[i], filtered_df['eps'].iloc[i], round(filtered_df['eps'].iloc[i], 2))
    plt.tight_layout()
    plt.savefig(f'{ticker}_eps_trend.png')
    plt.close()

    plt.figure(figsize=(6, 4))
    plt.plot(filtered_df['calendarYear'], filtered_df['ebitda'], marker='o', color='g')
    plt.title(f'{ticker} EBITDA Trend Over Last 5 Years')
    plt.xlabel('Year')
    plt.ylabel('EBITDA')
    
    for i in range(len(filtered_df)):
        plt.text(filtered_df['calendarYear'].iloc[i], filtered_df['ebitda'].iloc[i], round(filtered_df['ebitda'].iloc[i], 2))
    plt.tight_layout()
    plt.savefig(f'{ticker}_ebitda_trend.png')
    plt.close()

    plt.figure(figsize=(6, 4))
    plt.plot(filtered_df['calendarYear'], filtered_df['grossProfit'], marker='o', color='r')
    plt.title(f'{ticker} Gross Profit Trend Over Last 5 Years')
    plt.xlabel('Year')
    plt.ylabel('Gross Profit')
    
    for i in range(len(filtered_df)):
        plt.text(filtered_df['calendarYear'].iloc[i], filtered_df['grossProfit'].iloc[i], round(filtered_df['grossProfit'].iloc[i], 2))
    plt.tight_layout()
    plt.savefig(f'{ticker}_gross_profit_trend.png')
    plt.close()

In [4]:
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, self.title, 0, 1, 'C')

    def add_table(self, df):
        self.set_font('Arial', '', 10)
        col_width = self.w / 7
        self.ln(10)
        for col in df.columns:
            self.cell(col_width, 10, col, 1, 0, 'C')
        self.ln()
        for i in range(len(df)):
            for j in range(len(df.columns)):
                self.cell(col_width, 10, str(df.iloc[i, j]), 1, 0, 'C')
            self.ln()

    def add_image(self, image_path):
        self.add_page()
        self.image(image_path, x=10, y=30, w=190)

In [5]:
def generate_pdf(ticker, filtered_df):
    pdf = PDF()
    pdf.title = f'{ticker} Financial Statement'
    pdf.add_page()


    pdf.set_font('Arial', 'B', 12)
    pdf.cell(0, 10, f'{ticker} Financial Data', ln=True, align='L')
    pdf.add_table(filtered_df)


    pdf.add_image(f'{ticker}_eps_trend.png')
    pdf.add_image(f'{ticker}_ebitda_trend.png')
    pdf.add_image(f'{ticker}_gross_profit_trend.png')

    pdf.output(f'{ticker}_financial_statement.pdf')


API_KEY = '97nCa1Uxcak8UmXtbumbGdtneDNNo4Yj'

tickers = ['AAPL', 'MSFT']

for ticker in tickers:
    filtered_df = fetch_financial_data(ticker, API_KEY)
    generate_plots(filtered_df, ticker)
    generate_pdf(ticker, filtered_df)
